<a href="https://colab.research.google.com/github/Codilis/Pyspark-Projects/blob/master/SpotifyMusicData/Spotify_Data_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Data Source : https://www.kaggle.com/datasets/kapturovalexander/spotify-data-from-pyspark-course/data
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=f053e18d44e22d631e8c37c212a472f6cc9ccbe0d4466fd2b4b5dcadbf9510ed
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
import pyspark as sp
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from collections import defaultdict


In [4]:
# Global Functions
def rename_columns(df, new_names):
  for k, v in new_names.items():
    df = df.withColumnRenamed(k,v)
  return df


In [5]:
#Create my_spark
spark = SparkSession.builder.getOrCreate()

In [6]:
github_url = "https://raw.githubusercontent.com/Codilis/Pyspark-Projects/master/SpotifyMusicData/spotify-data/{i}.csv"
data_files = [github_url.format(i=i) for i in range(1, 5)]

pd_df = pd.read_csv(data_files[0])

for i in range(1, len(data_files)):
  temp = pd.read_csv(data_files[i], header=None, names=pd_df.columns.tolist())
  pd_df = pd.concat([pd_df, temp], axis=0)

pd_df.head()

,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],158648,1928,1928,0.995,0.708,0.1950,0.563,0.1510,-12.428,0.0506,118.469,0.7790,1,10,0,0
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",282133,1928,1928,0.994,0.379,0.0135,0.901,0.0763,-28.454,0.0462,83.972,0.0767,1,8,0,0
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],104300,1928,1928,0.604,0.749,0.2200,0.000,0.1190,-19.924,0.9290,107.177,0.8800,0,5,0,0
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],180760,9/25/28,1928,0.995,0.781,0.1300,0.887,0.1110,-14.734,0.0926,108.003,0.7200,0,1,0,0
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",687733,1928,1928,0.990,0.210,0.2040,0.908,0.0980,-16.829,0.0424,62.149,0.0693,1,11,1,0


In [7]:
spark_df = spark.createDataFrame(pd_df)
schema = ArrayType(StringType())
spark_df = spark_df.withColumn('artists', F.from_json(spark_df["artists"], schema))
spark_df.limit(5).show()

+--------------------+--------------------+--------------------+-----------+------------+----+------------+------------+------+----------------+--------+--------+-----------+-------+-------+----+---+----------+--------+
|                  id|                name|             artists|duration_ms|release_date|year|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|  tempo|valence|mode|key|popularity|explicit|
+--------------------+--------------------+--------------------+-----------+------------+----+------------+------------+------+----------------+--------+--------+-----------+-------+-------+----+---+----------+--------+
|6KbQ3uYMLKb5jDxLF...|Singende Bataillo...|   [Carl Woitschach]|     158648|        1928|1928|       0.995|       0.708| 0.195|           0.563|   0.151| -12.428|     0.0506|118.469|  0.779|   1| 10|         0|       0|
|6KuQTIu1KoTTkLXKr...|Fantasiestücke, O...|[Robert Schumann,...|     282133|        1928|1928|       0.994|       0.379|

## Artist Analysis

In [ ]:
# Get the names of all the artist from the dataset
distinct_artists = spark_df.select(F.explode(F.col('artists'))).distinct()

# Artists Average Song Duration, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, valence, popularity
new_names = {'avg(popularity)': 'AveragePopularity', 'avg(duration_ms)': 'AverageDuration_ms', 'avg(acousticness)': 'AverageAcousticness', 'avg(danceability)': 'AverageDanceability',
             'avg(energy)': 'AverageEnergy', 'avg(instrumentalness)': 'AverageInstrumentalness', 'avg(liveness)': 'AverageLiveness', 'avg(loudness)': 'AverageLoudness',
             'avg(speechiness)': 'AverageSpeechiness', 'avg(tempo)': 'AverageTempo', 'avg(valence)': 'AverageValence'}

artist_popularity = spark_df.select(
    F.explode(F.col('artists')).alias('artists'), F.col('popularity'), F.col('duration_ms'), F.col('acousticness'), F.col('danceability'), F.col('energy'), F.col('instrumentalness'),
    F.col('liveness'), F.col('loudness'), F.col('speechiness'), F.col('tempo'), F.col('valence')).groupBy(F.col('artists')).avg()
artist_popularity = rename_columns(artist_popularity, new_names)


# Most Active Year for an artist
most_active_year = spark_df.select(F.explode(F.col('artists')).alias('artists'), F.col('year')).groupBy(F.col('artists'), F.col('year')).count().select(
    F.col('artists'), F.col('year'), F.col('count'), F.row_number().over(Window.partitionBy("artists").orderBy(F.col("count").desc())).alias("RowNumber")).filter(
        "RowNumber = 1").select(F.col("artists"), F.col("year").alias('MostActiveYear'), F.col("count").alias("ActiveYearSongCount"))

# Number of Explicit Songs By an artist
explicit_song_count = spark_df.select(F.explode(F.col('artists')).alias('artists'), F.col('explicit')).groupBy(F.col('artists')).agg(F.sum("explicit")).alias('ExplicitCount')

# Most Popular Year
most_popular_year = spark_df.select(F.explode(F.col('artists')).alias('artists'), F.col('year'), F.when(F.col('popularity') == 0, 0).otherwise(1).alias('Popular')).groupBy(
    F.col('artists'), F.col('year')).agg(F.sum("Popular").alias("Count")).select(
    F.col('artists'), F.col('year'), F.col('count'), F.row_number().over(Window.partitionBy("artists").orderBy(F.col("count").desc())).alias("RowNumber")).filter(
        "RowNumber = 1").select(F.col("artists"), F.col("year").alias("PopularYear"), F.col("count").alias("SongsInTop100"))

# First and Last Song
artist_last_song = spark_df.select(F.explode(F.col('artists')).alias('artists'), F.col('name'), F.col('year'), F.col('release_date')).select(
                      F.col('artists'), F.col('name'), F.col('year'), F.col('release_date'),
                F.row_number().over(Window.partitionBy("artists").orderBy(F.col("year").desc(), F.col("release_date").desc())).alias("RowNumber")). \
                filter("RowNumber = 1").select(F.col("artists"), F.col("name").alias("LastSong"), F.col("year").alias("LastYear"))

artist_first_song = spark_df.select(F.explode(F.col('artists')).alias('artists'), F.col('name'), F.col('year'), F.col('release_date')).select(
                      F.col('artists'), F.col('name'), F.col('year'), F.col('release_date'),
                F.row_number().over(Window.partitionBy("artists").orderBy(F.col("year"), F.col("release_date"))).alias("RowNumber")). \
                filter("RowNumber = 1").select(F.col("artists"), F.col("name").alias("FirstSong"), F.col("year").alias("FirstYear"))


# Most Collabarated Artist
a = spark_df.select(F.explode(F.col('artists')).alias('Aartists'), F.col('name'))
b = spark_df.select(F.explode(F.col('artists')).alias('Bartists'), F.col('name'))
most_collaboration = a.join(b, "name", "inner").filter("Aartists != Bartists").groupBy("Aartists", "Bartists").count().\
  select(F.col("Aartists"), F.col("Bartists"), F.col("count"), F.row_number().over(Window.partitionBy("Aartists").orderBy(F.col("count").desc())).alias("RowNumber")).filter(
        "RowNumber = 1").select(F.col("Aartists").alias("artists"), F.col("Bartists").alias("Partner"), F.col("count").alias("ColabratedSongsCount"))


# distinct_artists.count(), artist_popularity.count(), most_active_year.count(), explicit_song_count.count(), most_popular_year.count(), artist_last_song.count(), artist_first_song.count(), most_collaboration.count()

artist_df = artist_popularity.join(most_active_year, 'artists', 'inner').join(explicit_song_count, 'artists', 'inner').join(most_popular_year, 'artists', 'inner').\
join(artist_first_song, 'artists', 'inner').join(artist_last_song, 'artists', 'inner').join(most_collaboration, 'artists', 'left')

artist_df.show(truncate=False)

+-----------------------------------------------------------------+------------------+------------------+-------------------+-------------------+-------------------+-----------------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------+-------------------+-------------+-----------+-------------+------------------------------------------------+---------+------------------------------------------------+--------+----------------------------------------------+--------------------+
|artists                                                          |AveragePopularity |AverageDuration_ms|AverageAcousticness|AverageDanceability|AverageEnergy      |AverageInstrumentalness|AverageLiveness    |AverageLoudness    |AverageSpeechiness |AverageTempo      |AverageValence     |MostActiveYear|ActiveYearSongCount|sum(explicit)|PopularYear|SongsInTop100|FirstSong                                       |FirstYear|LastSong                     

## Year Analysis

In [27]:
# Most Active artist for a year
most_active_artist = spark_df.select(F.explode(F.col('artists')).alias('artists'), F.col('year')).groupBy(F.col('artists'), F.col('year')).count().select(
    F.col('artists'), F.col('year'), F.col('count'), F.row_number().over(Window.partitionBy("year").orderBy(F.col("count").desc())).alias("RowNumber")).filter(
        "RowNumber = 1").select(F.col("year"), F.col("artists").alias('MostActiveArtist'), F.col("count").alias("SongsReleased"))

# MostPopularSong
most_popular_song = spark_df.filter("popularity > 0").select(F.col('year'), F.col("name"), F.col('popularity'), F.row_number().over(
    Window.partitionBy("year").orderBy(F.col('popularity'))).alias("RowNumber")).filter("RowNumber=1").select(F.col("year"), F.col('name').alias("PopularSong"))

# Average Song Duration
average_duration_by_year = spark_df.select(F.col('year'), F.col("duration_ms")).groupby(F.col("year")).agg({"duration_ms":"avg"}).select(
    F.col("year"), F.try_divide(F.col("avg(duration_ms)"), F.lit(1000)).alias("AverageSongDuration"))

# Number of artists active during a year
active_artists = spark_df.select(F.explode(F.col('artists')).alias('artists'), F.col('year')).distinct().groupBy(F.col('year')).count().select(F.col('year'), F.col('count').alias("ActiveArtistsCount"))

## Data Visualization

In [17]:
import plotly.express as px

In [32]:
# Trend in Average Song Duration
average_duration_plot = px.scatter(average_duration_by_year.toPandas(), x='year', y="AverageSongDuration")
average_duration_plot.show()

In [33]:
# Artists Active during a given year
active_artists_plot = px.scatter(active_artists.toPandas(), x='year', y="ActiveArtistsCount")
active_artists_plot.show()

+----------------+-------+-----+----------+
|          Artist|Average|Count|Popularity|
+----------------+-------+-----+----------+
| Johannes Brahms|    4.1|  313|      1282|
|    Morton Gould|    7.0|    8|        56|
|       B Jayamma|    0.0|   14|         0|
|Mixalis Thomakos|    0.0|   22|         0|
|      Alan Mills|   1.67|   18|        30|
+----------------+-------+-----+----------+

